In [5]:
import cv2
from keras.models import model_from_json
import numpy as np
from keras import losses

In [58]:
cap = cv2.VideoCapture(0)
while True:
    _,frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HLS)
    hsv = cv2.resize(hsv,(48,48))
    cv2.imshow("video",hsv)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [48]:
json_file = open('model3_VGG16.json','r')
loaded_json_model = json_file.read()
json_file.close()
cnn_model = model_from_json(loaded_json_model)

In [49]:
cnn_model.load_weights('model3_VGG16.h5')

In [50]:
cnn_model.compile(optimizer= 'adam',loss = losses.categorical_crossentropy,metrics = ['accuracy'])

In [11]:
alphabets = ('a','b','c','d','e','f','g','h','i','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y')

In [21]:
close_cascade = cv2.CascadeClassifier('closed_palm.xml')
open_cascade = cv2.CascadeClassifier('open_palm.xml')

In [67]:
def detect(gray, frame):
    gray = cv2.flip(gray,1)
    frame = cv2.flip(frame,1)
    hand1 = close_cascade.detectMultiScale(gray, 1.3, 3) #(x,y,w,h) tupple is returned
    for x,y,w,h in hand1:
        roi = frame[x-50:x+w+50,y-50:y+h+50]
        outline = cv2.rectangle(frame,(x-50,y-50),(x+w+50,y+h+50),(255,0,0),2)   
        try:
            hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
            hsv = cv2.resize(hsv,(48,48))
            hsv = hsv/255
            hsv_expanded = np.expand_dims(hsv,axis=0)
            detected_letter = str(alphabets[np.argmax(cnn_model.predict(hsv_expanded))])
            font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(outline,"FIST "+detected_letter,(x,y),font,1,(255,0,0),2,cv2.LINE_AA) 
        except:
            continue
    hand2 = open_cascade.detectMultiScale(gray, 1.2, 3) #(x,y,w,h) tupple is returned
    for x,y,w,h in hand2:
        roi = frame[x-50:x+w+50,y-50:y+h+50]
        outline = cv2.rectangle(frame,(x-50,y-50),(x+w+50,y+h+50),(255,0,0),2)
        try:
            hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
            hsv = cv2.resize(hsv,(48,48))
            hsv = hsv/255
            hsv_expanded = np.expand_dims(hsv,axis=0)
            detected_letter = str(alphabets[np.argmax(cnn_model.predict(hsv_expanded))])
            font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(outline,"PALM "+detected_letter,(x,y-10),font,1,(255,0,0),2,cv2.LINE_AA)
        except:
            continue
    return frame

In [68]:
cap = cv2.VideoCapture(0)
while True:
    _,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    canvas = detect(gray,frame)
    cv2.imshow('video',canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [68]:
cap.release()
cv2.destroyAllWindows()